In [8]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(100_000_000_000)

from calitp.tables import tbl
from calitp import query_sql
import calitp.magics
import branca

import shared_utils
import utils

from siuba import *
import pandas as pd
import geopandas as gpd
import shapely

import datetime as dt
import time
from zoneinfo import ZoneInfo

import rt_analysis2 as rt
import importlib

importlib.reload(utils)
importlib.reload(rt)

import gcsfs
fs = gcsfs.GCSFileSystem()

In [2]:
# analysis_date = dt.date(2022, 1, 11)
## all data incomplete -- did it crash? (nothing before 9ish)

In [4]:
analysis_date = dt.date(2022, 1, 12)

In [5]:
samtrans_itp_id = 290

In [4]:
# utils.get_vehicle_positions(samtrans_itp_id, analysis_date)

In [5]:
# utils.get_trips(samtrans_itp_id, analysis_date)

In [6]:
# utils.get_stop_times(samtrans_itp_id, analysis_date, force_clear=False)

In [7]:
# utils.get_stops(samtrans_itp_id, analysis_date)

In [8]:
# sam_trips >> head(5)

In [9]:
# sam_vp >> head(5)

In [10]:
# sam_vp >> inner_join(_, sam_trips, on ='trip_id')

In [11]:
## TODO
# lbt_analysis.position_interpolators['9288623']['rt'].trip_key
# generated 1553149201202261125 on 11/30/21 but this doesn't seem to exist?

In [6]:
air_joined = pd.read_parquet('airtable_joined.parquet')

In [7]:
air_joined

,Name,calitp_itp_id,Reporting Category,Caltrans District,GTFS Static Status,rt_status,Missing Static,Services Needing Alerts,Services Needing TripUpdates or VehiclePositions,calitp_url_number,min_timestamp,max_timestamp
0,Long Beach Transit,170,Core,07 - Los Angeles,Static Incomplete,RT Incomplete,AquaLink,"Galaxy Express, AquaLink, UCLA/Westwood Commut...","Galaxy Express, AquaLink, UCLA/Westwood Commut...",0,2021-07-08 20:58:59,2022-01-13 15:37:53
1,Peninsula Corridor Joint Powers Board,246,Core,04 - Oakland,Static Incomplete,RT Incomplete,"Caltrain Shuttles, Caltrain Millbrae Express","Caltrain Shuttles, Caltrain Millbrae Express","Caltrain Shuttles, Caltrain Millbrae Express",1,2021-12-29 14:34:19,2022-01-13 15:38:22
2,City of Visalia,361,Core,06 - Fresno,Static Incomplete,RT Incomplete,V-LINE,"V-LINE, Sequoia Shuttle, Visalia Dial-A-Ride","V-LINE, Sequoia Shuttle, Visalia Dial-A-Ride",0,2015-01-03 02:55:42,2022-01-13 15:38:35
3,San Mateo County Transit District,290,Core,04 - Oakland,Static Incomplete,RT OK,"Bayshore Brisbane Senior Shuttle, SamTrans Shu...","Redi-Wheels, RediCoast, Bayshore Brisbane Seni...","Redi-Wheels, RediCoast, Bayshore Brisbane Seni...",1,2021-12-29 14:34:05,2022-01-13 15:38:11
4,Marin County Transit District,194,Core,04 - Oakland,Static Incomplete,RT OK,Muir Woods Shuttle,"Marin Access Paratransit, Muir Woods Shuttle, ...","Marin Access Paratransit, Muir Woods Shuttle, ...",0,2015-01-01 16:55:46,2022-01-13 15:38:36
5,Monterey-Salinas Transit,208,Core,05 - San Luis Obispo,Static Incomplete,RT OK,MST Senior Shuttles,"MST Monterey Trolley, MST RIDES, MST Taxi Vouc...","MST Monterey Trolley, MST RIDES, MST Taxi Vouc...",0,2021-07-08 20:57:45,2022-01-13 15:38:25
6,Napa Valley Transportation Authority,218,Core,04 - Oakland,Static OK,RT Incomplete,None,"Calistoga Shuttle, Taxi Scrip, Vine Go, Americ...","Calistoga Shuttle, Taxi Scrip, Vine Go, Americ...",1,2021-12-29 14:34:17,2022-01-13 15:38:19
7,"Golden Gate Bridge, Highway and Transportation...",127,Core,04 - Oakland,Static OK,RT Incomplete,None,"Golden Gate Ferry, Golden Gate Transit Paratra...","Golden Gate Ferry, Golden Gate Transit Paratra...",1,2021-12-29 14:34:12,2022-01-13 15:38:12
8,Transit Joint Powers Authority for Merced County,343,Core,10 - Stockton,Static OK,RT Incomplete,None,"Merced The Micro Bus, Merced The Bus Paratransit","Merced The Bus, Merced The Micro Bus, Merced T...",0,2021-12-21 13:50:26,2022-01-13 15:38:37
9,Sacramento Regional Transit District,273,Core,03 - Marysville,Static OK,RT Incomplete,None,"SacRT Light Rail, e-van, SacRT GO, Elk Grove T...","e-van, SacRT GO, SacRT Neighborhood Ride",0,2021-12-21 14:04:00,2022-01-13 15:38:00


In [8]:
for agency in air_joined.calitp_itp_id:
    try:
        analysis = rt.OperatorDayAnalysis(agency, analysis_date)
    except Exception as e:
        print(f'failed on agency {agency}')
        print(e)

found parquet
found parquet
found parquet
found parquet
found_parquet
9289275
sc_called
check location data for trip -4329841926259451152
sc_called
(39, 12)
9288197
sc_called
check location data for trip 477904054299874101
sc_called
(70, 12)
check location data for trip 477904054299874101
sc_called
(65, 12)
check location data for trip 477904054299874101
sc_called
(63, 12)
9288671
sc_called
9288672
sc_called
check location data for trip -4332510122068877721
sc_called
(73, 12)
9288276
sc_called
check location data for trip -8407187321077433825
sc_called
(47, 12)
check location data for trip -8407187321077433825
sc_called
(43, 12)


/opt/conda/lib/python3.9/site-packages/siuba/sql/utils.py:52: SAWarning: Dialect bigquery:bigquery will not make use of SQL compilation caching as it does not set the 'supports_statement_cache' attribute to ``True``.  This can have significant performance implications including some performance degradations in comparison to prior SQLAlchemy versions.  Dialect maintainers should seek to set this attribute to True after appropriate development and testing for SQLAlchemy 1.4 caching support.   Alternatively, this attribute may be set to False which will disable this warning. (Background on this error at: https://sqlalche.me/e/14/cprf)
  return self.connectable.execute(*args, **kwargs)


gs://calitp-analytics-data/data-analyses/rt_delay/cached_views/
stops_246_2022-01-12


/home/jovyan/data-analyses/_shared_utils/shared_utils/utils.py:36: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a production setting unless you are able to rewrite your Parquet/Feather files.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')
  gdf.to_parquet(f"./{FILE_NAME}.parquet")
/home/jovyan/data-analyses/_shared_utils/shared_utils/utils.py:36: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises

failed on agency 246
routelines must not be empty
gs://calitp-analytics-data/data-analyses/rt_delay/cached_views/
stops_361_2022-01-12


...

failed on agency 226
routelines must not be empty


/home/jovyan/data-analyses/rt_delay/utils.py:140: UserWarning: rt data starts after analysis date
  warnings.warn('rt data starts after analysis date')


gs://calitp-analytics-data/data-analyses/rt_delay/cached_views/
stops_259_2022-01-12


/home/jovyan/data-analyses/_shared_utils/shared_utils/utils.py:36: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a production setting unless you are able to rewrite your Parquet/Feather files.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')
  gdf.to_parquet(f"./{FILE_NAME}.parquet")
/home/jovyan/data-analyses/_shared_utils/shared_utils/utils.py:36: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises

failed on agency 259
routelines must not be empty


In [ ]:
## fix routelines thing...

In [26]:
# metro_analysis.position_interpolators['10105002581045-DEC21']['rt'].detailed_speed_map()

In [27]:
# lbt_analysis.position_interpolators['9288672']['rt'].detailed_speed_map()

In [6]:
sam_analysis = rt.OperatorDayAnalysis(samtrans_itp_id, analysis_date)

found parquet
found parquet
found parquet
found parquet
found_parquet
11031698-136-Blocks-Weekday-54
sc_called
check location data for trip -1949207877340033680
sc_called
(139, 12)
check location data for trip -1949207877340033680
sc_called
(136, 12)
check location data for trip -1949207877340033680
sc_called
(134, 12)
check location data for trip -1949207877340033680
sc_called
(132, 12)
check location data for trip -1949207877340033680
sc_called
(130, 12)
check location data for trip -1949207877340033680
sc_called
(128, 12)
check location data for trip -1949207877340033680
sc_called
(126, 12)
check location data for trip -1949207877340033680
sc_called
(124, 12)
check location data for trip -1949207877340033680
sc_called
(122, 12)
check location data for trip -1949207877340033680
sc_called
(120, 12)
check location data for trip -1949207877340033680
sc_called
(118, 12)
check location data for trip -1949207877340033680
sc_called
(116, 12)
check location data for trip -1949207877340033680

In [ ]:
rt.OperatorDayAnalysis._generate_stop_delay_view(sam_analysis)

insufficient bounding points for trip -4321367742262557686, location 0.0: start/end of route?
insufficient bounding points for trip -4321367742262557686, location 363.46020677282127: start/end of route?


/opt/conda/lib/python3.9/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [45]:
## TODO '11031623-136-Blocks-Weekday-54' should not be eastbound...

In [198]:
# example_vp_trip.detailed_speed_map()

## To-do online

* test new getters
* verify info columns, drop and add to joins as needed
* investigate 294 bug, time-permitting
* ...
* run for all existing operators... (reality check results...)